# Artificial Neural Network

The purpose of the following notebook is to provide a simple implementation to get hands on experience, and thus hopefully also a deeper understanding of the backwards error propagation approach to neural network. The task for the network is to classify iris set of data from [UCI](https://archive.ics.uci.edu/ml/datasets/Iris), and will thereby be a form of supervised learning.


## 1. Setting up the artificial network class [WIP]


In [252]:
import numpy as np
import pandas as pd


In [253]:
def activation_function(x):
    """Calculates the activation of a node give an input

    Args:
        x (integer: Scalar) :  Number representing the sum of the input to the give node
    """
    return (1 + np.e**-x)**-1


def der_activation_function(x):

    return (np.e**-x)/((1+np.e**-x)**2)


activation = np.vectorize(activation_function)
der_activation = np.vectorize(der_activation_function)


In [254]:
def cost_function(prediction, solution):
    """Calculates the cost of a single sample

    Args:
        prediction (ndarray) : Output from the learner
        soution (ndarray) : Solution for the sample

    Returns: 
        float : Cost of the single training sample
    """
    return np.sum((prediction - solution)**2)


In [303]:
class ANN:
    def __init__(self, features, hidden_layers, outputs, learning_rate=0.01):
        """
        Takes as input
            hidden_layers : 1d array with number of neurons to include in each of the hidden layers
            features : int representing the number of features of the data set - corresponds to the first (input) layer
            outputs : int representing the number of possible outputs - and thereby number of nodes in the final (output) layer
        """

        # Meta data
        self.num_layers = len(hidden_layers) + 2
        self.num_outputs = outputs
        self.num_features = features
        self.learning_rate = learning_rate

        combined_layers = np.array([features] + hidden_layers + [outputs])

        # Setting up the layers
        self.layers = []
        self.deltas = []
        for num in combined_layers:
            self.layers.append([0 for _ in range(num)])
            self.deltas.append([0 for _ in range(num)])

        # Setting up weights and layer inputs
        self.layer_inputs = []
        self.weights = []
        #self.margins = []
        for i in range(len(combined_layers)-1):
            self.weights.append(np.random.rand(
                combined_layers[i], combined_layers[i+1]))
            #self.margins.append(np.zeros(np.shape((combined_layers[i], combined_layers[i+1]))))
            self.layer_inputs.append([0 for _ in range(combined_layers[i])])

    def forward_pass(self, sample):
        """Fixes the neural network to a sample and generates the prediction at the output layer

        Args:
            sample (ndarray): Array of floats which represents the value of the input sample
        """
        self.layers[0] = sample

        # Iterate through each layer and calculate activation based on
        for i in range(1, self.num_layers):
            # Fetch the activation and weight of the previous layers
            prev_activation = self.layers[i-1]
            weights = self.weights[i-1]

            # Calculate inputs of the current nodes based on activation of previous and weights between them
            self.layer_inputs[i-1] = np.dot(weights.T, prev_activation)

            # Update current layer with activation of inputs
            self.layers[i] = activation(self.layer_inputs[i-1])

    def predict(self, sample):
        """Predicts the label of a sample x

        Args:
            sample (ndarray): Array of integers which represents the first input layer of the network

        Returns:
            ndarray: Array of floats which represent the confidences of the classification for each category of the output
        """
        self.forward_pass(sample)
        return self.layers[-1]

    def backprops_error(self, solution):
        """Compares a prediction to a solution and calculates the error between the layer and the expected values
        (Propagates the deltas down the network from the output)

        Args:
            prediction (ndarray): Prediction, output of the network for some sample x
            solution (ndarray): The solution to the sample fetched from the dataset
        """
        
        # Calculate error of output layer
        self.deltas[-1] = der_activation(self.layer_inputs[-1]) * (self.layers[-1] - solution)

        # Iterate from outputlayer backwards and propagate error
        for i in range(len(self.weights) - 1, 0, -1):
            self.deltas[i] = der_activation(self.layer_inputs[i-1]) * np.dot(self.weights[i], self.deltas[i + 1])

            #self.margins[i] = self.learning_rate * self.layers[i] * der_activation(self.layer_inputs[i]) * self.deltas[i + 1]

    def update_network(self, solution):
        """Calculates the margins of weights for the network given a traning example, and updates the weights accordingly
        """
        for i in range(len(self.weights)):
            margins = -self.learning_rate * np.outer(self.layers[i], self.deltas[i + 1].T)
            self.weights[i] = self.weights[i] + margins

    def train_sample(self, sample, solution): 
        """Trains the network on a single sample

        Args:
            sample (ndarray(4,)): Sample to make prediction on
            solution (ndarray(3,)): Solution to the respective sample passed
        """
        self.predict(sample)
        self.backprops_error(solution)
        self.update_network(solution)

    def train(self, samples, solutions):
        """Trains the network on a set of samples and respective solutions

        Args:
            samples (ndarray(n,)): Set of samples for the network to make predictions on and train for
            solutions (ndarray(n,)): Set of respective solutions to the samples
        """
        assert len(samples) == len(solutions)

        for sample, solution in zip(samples, solutions):
            self.train_sample(sample, solution)

    


## Preprocessing data

The data is loaded from `./data/iris.data` and utilizes `pandas` to preprocess it. This includes splitting the attributes and the labels, and converting the categorical label to integers.


In [304]:
# Loading from file
df = pd.read_csv('./data/iris.data',
                 names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'label'])
y = df['label']
x = df.drop(['label'], axis=1)


In [305]:
# Converting labels from string to integers representing categories
uniques = y.unique()
label_map = {i: uniques[i] for i in range(len(uniques))}
y.replace(uniques, [i for i in range(len(uniques))], inplace=True)


## Setting up network and making predictions [WIP]

With the data loaded, the model is ready to be set up and make predictions based on the provided samples from the dataset.


In [306]:
# Setting up network based on metadata
model_hidden_layers = [10, 6]

ann = ANN(len(x.columns), model_hidden_layers, len(uniques))


In [307]:
len(ann.layer_inputs)


3

In [308]:
ann.layer_inputs


[[0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]

In [309]:
# Creating arrays with solutions for easier error calculation
solutions = []

for _, solution in y.items():

    solutions.append([1 if i == solution else 0 for i in range(len(uniques))])


In [310]:
ann.deltas

[[0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0]]

In [311]:
# Making predictions for each sample and printing error terms
predictions = []

for idx, sample in x.iterrows():
    pred = ann.predict(sample.to_numpy())
    ann.backprops_error(solutions[idx])
    ann.update_network(solutions[idx])
    print(cost_function(pred, solutions[idx]))


1.7968861450412419
1.796169826677648
1.7955173869642462
1.7950056084818578
1.794530482929604
1.794230320389647
1.7932536069758611
1.7927500087857888
1.7918517517870942
1.791422628251286
1.7910373108948034
1.790318292898629
1.789469711134056
1.788485679270027
1.788513605993717
1.7881064196890233
1.7872787990548342
1.7865483292068447
1.7861761829745446
1.7853622873346544
1.784762901654878
1.7840766400819994
1.7829212506386674
1.7828192878485272
1.7821269101308177
1.781283606291617
1.780746812647533
1.780012228633805
1.7792634598315562
1.778555650810197
1.7778710271333273
1.7773519913242737
1.7766459637387177
1.775993987216535
1.7750123560265523
1.7741994937040162
1.7737111607462455
1.7728801045236229
1.7719239614433586
1.7715725271819334
1.7707659313377255
1.7696332672226935
1.7690511576221353
1.7688068465229256
1.7681933843709086
1.7669820704721846
1.7665090673987103
1.7654256787875795
1.764957995087586
1.764007490144024
1.7813985467709919
1.7806545215872327
1.7799459223550576
1.7790668